# Analyze HTML GUI Prototypes: Console Error & Warning Counter


In [13]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [14]:
# Path to your generated_guis folder
generated_guis_path = '../generated_guis'
# List of app IDs and method names (adjust as needed)
app_ids = ['12740', '14283', '18782', '20947', '22151', '27360', '27382', '27707', '30982', '31390', '32310', '3261', '33383', '34346', '34517', '34527', '35526', '3727', '37505', '38961', '40673', '43872', '43977', '44756', '47926', '49794', '53054', '53469', '54377', '54468', '56905', '58124', '59429', '59576', '61851', '63575', '64858', '65592', '67044', '68368', '69574', '69587', '70410', '8640']
method_names = ['instruction', 'pd_zs', 'pd_fs', 'ref_instruction']

In [15]:
# Prepare list of HTML file paths and metadata
prototype_files = []
for app_id in app_ids:
    for method in method_names:
        html_path = os.path.join(generated_guis_path, str(app_id), f'{method}.html')
        if os.path.isfile(html_path):
            prototype_files.append({'app_id': app_id, 'method': method, 'path': html_path})

In [16]:
# Selenium Chrome options for local files
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run headless (no UI)
chrome_options.add_argument('--allow-file-access-from-files')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--log-level=3')
chrome_options.set_capability('goog:loggingPrefs', {'browser': 'ALL'})

In [17]:
results = []
driver = webdriver.Chrome(options=chrome_options)

In [18]:
for proto in prototype_files:
    file_url = 'file://' + os.path.abspath(proto['path'])
    driver.get(file_url)

    # Wait for JS to execute (can adjust time)
    driver.implicitly_wait(2)

    logs = driver.get_log('browser')
    error_messages = [log['message'] for log in logs if log['level'] == 'SEVERE']
    warning_messages = [log['message'] for log in logs if log['level'] == 'WARNING']
    results.append({
        'app_id': proto['app_id'],
        'method': proto['method'],
        'errors': len(error_messages),
        'warnings': len(warning_messages),
        'error_descriptions': ' || '.join(error_messages),
        'warning_descriptions': ' || '.join(warning_messages)
    })

In [19]:
driver.quit()

In [22]:
# Display results as table
df = pd.DataFrame(results)
df

,app_id,method,errors,warnings,error_descriptions,warning_descriptions
0,12740,instruction,0,0,,
1,12740,pd_zs,0,0,,
2,12740,pd_fs,0,0,,
3,12740,ref_instruction,0,0,,
4,14283,instruction,2,0,https://via.placeholder.com/40 - Failed to loa...,
...,...,...,...,...,...,...
171,70410,ref_instruction,0,0,,
172,8640,instruction,0,0,,
173,8640,pd_zs,0,0,,
174,8640,pd_fs,0,0,,


In [21]:
# Save to CSV
df.to_csv('prototypes_console_errors.csv', index=False, sep=';')

In [31]:
df = pd.DataFrame(results)
agg_errors = df.groupby('app_id', as_index=False)['errors'].sum()
agg_errors_sorted = agg_errors.sort_values(by='errors', ascending=False)

agg_errors_sorted

,app_id,errors
28,54377,26
27,53469,24
5,27360,23
34,61851,16
13,34346,14
35,63575,9
16,35526,9
25,49794,8
14,34517,6
17,3727,5
